Understanding GPT by Julia -- Materials
=======================================

**Date:** [2023-06-17 Sat 22:27]



## Get all available models



In [1]:
using OpenAI
secret_key = "your openai api key"
ENV["HTTP_PROXY"] = "your proxy" 
ENV["HTTPS_PROXY"] = "your proxy" 
r = list_models(secret_key).response["data"]
# println(r)

-   Convert the list to a dictionary



In [1]:
dict_model = Dict()
for model in r
    dict_model[model.id] = Dict("object" => model.object,
                                "created" => model.created, 
                                "owned_by" => model.owned_by, 
                                "permission" => model.permission, 
                                "root" => model.root,
                                "parent" => model.parent)
end

## Get the probability distribution of the next word



### Get the response from OpenAI



In [1]:
response = create_completion(
    secret_key,
    "text-davinci-003",
    prompt="My wife is always", 
    n = 100,
    logprobs = 5,
    max_tokens = 1
)

### Calculate the frequency of each word (word-level probabilities)



In [1]:
empirical_distribution = Dict();
model_distribution = Dict();

In [1]:
for word in response.response.choices
    empirical_distribution[word["text"]] = get(empirical_distribution, word["text"], 0) + 1
    if word["text"] ∉ keys(model_distribution)
        model_distribution[word["text"]] = exp(word["logprobs"]["token_logprobs"][1])
    end
end

-   Visualization



In [1]:
using DataStructures
sorted_empirical_distribution = OrderedDict(sort(collect(empirical_distribution), by = tuple -> last(tuple), rev=true))
sorted_model_distribution = OrderedDict(sort(collect(model_distribution), by = tuple -> last(tuple), rev=true));

In [1]:
using StatsBase
using Plots
# Calculate frequencies
bar(collect(keys(sorted_empirical_distribution)), collect(values(sorted_empirical_distribution)), legend = false,
    xticks = (1:length(sorted_empirical_distribution), collect(keys(sorted_empirical_distribution))), xrotation = 45)
title!("My wife is always ...")
ylabel!("Empirical Frequency")

#+attr_org: :width 1140

In [1]:
using StatsBase
using Plots
# Calculate frequencies
bar(collect(keys(sorted_model_distribution)), collect(values(sorted_model_distribution)), legend = false,
    xticks = (1:length(sorted_model_distribution), collect(keys(sorted_model_distribution))), xrotation = 45)
title!("My wife is always ...")
ylabel!("Model Probability")

#+attr_org: :width 1140

### Probe the effect of temperature



#### Zero temperature



In [1]:
response_zero_temperature = create_completion(
    secret_key,
    "text-davinci-003",
    prompt="My wife is always", 
    n = 100,
    logprobs = 5,
    max_tokens = 1,
    temperature = 0,            # Set temperature to zero
);

-   Empirical distribution



In [1]:
empirical_distribution_zero_temperature = Dict();
model_distribution_zero_temperature = Dict();

In [1]:
for word in response_zero_temperature.response.choices
    empirical_distribution_zero_temperature[word["text"]] = get(empirical_distribution_zero_temperature, word["text"], 0) + 1
    if word["text"] ∉ keys(model_distribution)
        model_distribution_zero_temperature[word["text"]] = exp(word["logprobs"]["token_logprobs"][1])
    end
end

In [1]:
empirical_distribution_zero_temperature

Dict{Any, Any} with 1 entry:
  " telling" => 100

-   Model distribution (there is no distributions)



In [1]:
# model_distribution_zero_temperature = response_zero_temperature.response.choices[1]["logprobs"]["top_logprobs"][1]
for word in keys(response_zero_temperature.response.choices[1]["logprobs"]["top_logprobs"][1])
    model_distribution_zero_temperature[string(word)] = exp(response_zero_temperature.response.choices[1]["logprobs"]["top_logprobs"][1][word])
end
# keys(Dict(model_distribution_zero_temperature))
sorted_model_distribution_zero_temperature = OrderedDict(sort(collect(model_distribution_zero_temperature), by = tuple -> last(tuple), rev=true));

In [1]:
using StatsBase
using Plots
# Calculate frequencies
bar(collect(keys(sorted_model_distribution_zero_temperature)), collect(values(sorted_model_distribution_zero_temperature)), legend = false,
    xticks = (1:length(sorted_model_distribution_zero_temperature), collect(keys(sorted_model_distribution_zero_temperature))), xrotation = 45)
title!("My wife is always ...")
ylabel!("Model Probability")

#+attr_org: :width 1140

#### Maximum temperature



In [1]:
response_max_temperature = create_completion(
    secret_key,
    "text-davinci-003",
    prompt="My wife is always", 
    n = 100,
    logprobs = 5,
    max_tokens = 1,
    temperature = 2,            # Set temperature to zero
);

In [1]:
empirical_distribution_max_temperature = Dict();
model_distribution_max_temperature = Dict();

In [1]:
for word in response_max_temperature.response.choices
    empirical_distribution_max_temperature[word["text"]] = get(empirical_distribution, word["text"], 0) + 1
    if (word["text"] ∉ keys(model_distribution_max_temperature)) && (word["text"] ∈ keys(model_distribution))
        model_distribution_max_temperature[word["text"]] = exp(word["logprobs"]["token_logprobs"][1])
    end
end

-   Visualization



In [1]:
using DataStructures
sorted_empirical_distribution_max_temperature = OrderedDict(sort(collect(empirical_distribution_max_temperature), by = tuple -> last(tuple), rev=true))
sorted_model_distribution_max_temperature = OrderedDict(sort(collect(model_distribution_max_temperature), by = tuple -> last(tuple), rev=true));

In [1]:
using StatsBase
using Plots
# Calculate frequencies
bar(collect(keys(sorted_empirical_distribution_max_temperature)), collect(values(sorted_empirical_distribution_max_temperature)), legend = false)
    # xticks = (1:length(sorted_empirical_distribution_max_temperature), collect(keys(sorted_empirical_distribution_max_temperature))), xrotation = 45)
title!("My wife is always ...")
ylabel!("Empirical Frequency")

#+attr_org: :width 1143

In [1]:
using StatsBase
using Plots
# Calculate frequencies
bar(collect(keys(sorted_model_distribution_max_temperature)), collect(values(sorted_model_distribution_max_temperature)), legend = false,
    xticks = (1:length(sorted_model_distribution_max_temperature), collect(keys(sorted_model_distribution_max_temperature))), xrotation = 45)
title!("My wife is always ...")
ylabel!("Model Probability")

#+attr_org: :width 1146

## Trajectory of the generating process



### One trajectory



-   Generate an entire sentence



In [1]:
response_single_sentence = create_completion(
    secret_key,
    "text-davinci-003",
    prompt="Physics is", 
    n = 1,
    logprobs = 5,
    max_tokens = 30,
    temperature = 1
);

In [1]:
response_single_sentence.response.choices[1]["text"]

" the natural science that involves the study of matter and its motion through space and time, along with related concepts such as energy and force. More broadly,"

-   For each generated word, get the embedding vector



In [1]:
splited_text = split(response_single_sentence.response.choices[1]["text"])
reconstruction = ["Physics is"]

for word in splited_text
    push!(reconstruction, string(reconstruction[end]," ", word))
end

-   Get the embeddings



In [1]:
embeddings_single_sentence = create_embeddings(secret_key,
                                               reconstruction);

In [1]:
embeddings_single_sentence_array = []

for embedding in embeddings_single_sentence.response["data"]
    push!(embeddings_single_sentence_array,embedding["embedding"])
end

In [1]:
using MultivariateStats
using TSne
matrix_embeddings = hcat(embeddings_single_sentence_array...)
# Perform t-SNE
Y = tsne(matrix_embeddings') # Y is now a 2x500 matrix
# Create scatter plot
plot(Y[:, 1], Y[:, 2], legend=false, line_z=1:length(Y), colorbar=true)
title!("Trajectory of 'Physics is ...'")
xlabel!("x-dimension")
ylabel!("y-dimension")

#+attr_org: :width 1140